In [2]:
import os
import json
import random
import numpy as np
import cv2 

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, datasets
from tqdm import tqdm
import copy
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2 

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, classification_report
)

# ⭐️ CONFIGURATION ⭐️
NUM_CLASSES = 4 

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# ================================
# 3. Transform / Dataset / DataLoader (4가지 클래스에 맞게 수정)
# ================================

train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    ToTensorV2(),
])


# ⭐️⭐️⭐️ 수정된 부분 ⭐️⭐️⭐️
# ImageFolder의 루트는 'train', 'val', 'test'를 포함하는 'img' 폴더가 되어야 합니다.
data_root = "/workspace/merge_data/data/img"

# CustomDataset 정의 (이전과 동일)
class CustomMultiClassDataset(Dataset):
    def __init__(self, full_dataset, transform):
        self.samples = full_dataset.samples 
        self.classes = full_dataset.classes 
        self.transform = transform
            
    def __len__(self):
        return len(self.samples)
            
    def __getitem__(self, idx):
        path, label = self.samples[idx]
            
        image = cv2.imread(path)
            
        if image is None:
            raise FileNotFoundError(f"Image not found or invalid: {path}")

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            augmented = self.transform(image=image)
            image = augmented['image']
            
        return image, torch.tensor(label, dtype=torch.long)


# ImageFolder는 data_root/train, data_root/val 폴더를 각각 로드합니다.
train_full_dataset = datasets.ImageFolder(os.path.join(data_root, "train"), transform=None) 
val_full_dataset = datasets.ImageFolder(os.path.join(data_root, "val"), transform=None) 

train_dataset = CustomMultiClassDataset(train_full_dataset, train_transform) 
val_dataset = CustomMultiClassDataset(val_full_dataset, val_transform) 

batch_size = 32
num_workers = 2 

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

print(f"Train classes (indices): {train_full_dataset.class_to_idx}")
print(f"Train samples: {len(train_dataset)}")
print(f"Val samples: {len(val_dataset)}")
print("\n✅ 전처리 및 데이터 로드 완료 (4가지 클래스).")
# ================================
# 4. 모델 정의 (ResNet18, 4-class) - 변경 없음
# ...
# ================================
# 사전학습된 resnet18
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# 마지막 fc를 4클래스로 교체 
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, NUM_CLASSES) 

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)


# ================================
# 5. 학습 함수 - 변경 없음
# ...
# ================================
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=20):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 30)

        # ---- Train ----
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total = 0

        for inputs, labels in tqdm(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

        epoch_loss = running_loss / total
        epoch_acc = running_corrects.double() / total
        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # ---- Validation ----
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        val_total = 0

        with torch.inference_mode():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)
                val_total += labels.size(0)

        val_loss /= val_total
        val_acc = val_corrects.double() / val_total
        print(f"Val   Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

        # best model 저장
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, "best_resnet_4_classes.pth") 
            print(">> best model updated")

    print(f"\nBest val Acc: {best_val_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model


# ================================
# 6. 학습 실행
# ================================
num_epochs = 10
model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs)


# ================================
# 7. 성능 지표 (val셋을 평가용으로 사용)
# ================================
model.eval()
all_labels = []
all_preds = []

with torch.inference_mode():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

acc  = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds, average='macro', zero_division=0)
rec  = recall_score(all_labels, all_preds, average='macro', zero_division=0)
f1   = f1_score(all_labels, all_preds, average='macro', zero_division=0)
cm   = confusion_matrix(all_labels, all_preds)

TARGET_NAMES = ["anger", "happy", "panic", "sadness"]

print("\n=== Val Metrics (Multi-class: 4 emotions) ===")
print(f"Accuracy (Overall) : {acc:.4f}")
print(f"Precision (Macro)  : {prec:.4f}")
print(f"Recall (Macro)     : {rec:.4f}")
print(f"F1-score (Macro)   : {f1:.4f}")
print("\nConfusion Matrix (rows: true, cols: pred)")
print(cm)

print("\nClassification report:")
print(classification_report(all_labels, all_preds, target_names=TARGET_NAMES))

device: cuda
Train classes (indices): {'anger': 0, 'happy': 1, 'panic': 2, 'sadness': 3}
Train samples: 5841
Val samples: 1151

✅ 전처리 및 데이터 로드 완료 (4가지 클래스).

Epoch 1/10
------------------------------


  0%|          | 0/183 [00:00<?, ?it/s]Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
  4%|▍         | 8/183 [00:07<02:20,  1.24it/s]Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
  5%|▌         | 10/183 [00:09<02:13,  1.29it/s]Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
  9%|▉         | 17/183 [00:15<02:26,  1.13it/s]Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
 13%|█▎        | 23/183 [00:21<02:44,  1.03s/it]Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
 16%|█▌        | 29/183 [00:26<02:17,  1.12it/s]Invalid SOS parameters for sequent

Train Loss: 0.8107 Acc: 0.6646



Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SO

Val   Loss: 0.6572 Acc: 0.7628
>> best model updated

Epoch 2/10
------------------------------


...

Train Loss: 0.0347 Acc: 0.9887



Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SO

Val   Loss: 1.0287 Acc: 0.7724

Best val Acc: 0.7889


Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS


=== Val Metrics (Multi-class: 4 emotions) ===
Accuracy (Overall) : 0.7889
Precision (Macro)  : 0.7876
Recall (Macro)     : 0.7871
F1-score (Macro)   : 0.7863

Confusion Matrix (rows: true, cols: pred)
[[212  17  32  30]
 [  7 279   7   6]
 [ 36  19 209  16]
 [ 41  15  17 208]]

Classification report:
              precision    recall  f1-score   support

       anger       0.72      0.73      0.72       291
       happy       0.85      0.93      0.89       299
       panic       0.79      0.75      0.77       280
     sadness       0.80      0.74      0.77       281

    accuracy                           0.79      1151
   macro avg       0.79      0.79      0.79      1151
weighted avg       0.79      0.79      0.79      1151



In [1]:
import sys
import torch
import torchvision
import numpy
import sklearn
import PIL
import tqdm

print("===========================================")
print("=== 🛠️ 라이브러리 버전 정리 (요약) ===")
print(f"Python: \t{sys.version.split()[0]}")
print("--- 딥러닝 코어 ---")
print(f"torch: \t\t{torch.__version__}")
print(f"torchvision: \t{torchvision.__version__}")
try:
    print(f"CUDA: \t\t{torch.version.cuda}")
except AttributeError:
    print("CUDA: \t\tN/A (CPU only or version info unavailable)")
print("--- 데이터/유틸리티 ---")
print(f"numpy: \t\t{numpy.__version__}")
print(f"sklearn: \t{sklearn.__version__}")
print(f"Pillow (PIL): \t{PIL.__version__}")
print(f"tqdm: \t\t{tqdm.__version__}")
print("===========================================")

=== 🛠️ 라이브러리 버전 정리 (요약) ===
Python: 	3.10.19
--- 딥러닝 코어 ---
torch: 		2.9.1+cu128
torchvision: 	0.24.1+cu128
CUDA: 		12.8
--- 데이터/유틸리티 ---
numpy: 		2.2.6
sklearn: 	1.7.2
Pillow (PIL): 	12.0.0
tqdm: 		4.67.1
